# Unified Pipeline for Image Preparation - FINAL FIXED VERSION

## Key Fixes in This Version:

### Step 6 - Continuous Global Numbering:
- Numbering continues across ALL images (not restarting for each image)
- Format: `00000001_filename.png` (8-digit index at the beginning)

### Step 7 - FIXED Black Detection and File Handling:
- **Cleans old format files** first (removes any leftover files from previous runs)
- **Properly detects black masks** by checking if all RGB values are (0,0,0)
- **Only processes new format files** (8-digit prefix)
- **Two-phase cleaning**:
  1. Remove completely black masks
  2. Remove non-matching snippets from orthos/normalmaps

# 1. Configuration Cell - Set All Paths and Parameters Here

In [8]:
import os

# === MAIN CONFIGURATION ===
# Base directory containing your project
BASE_DIR = "C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX"

# Input paths
COCO_JSON_PATH = os.path.join(BASE_DIR, "2025-08-10.json")
ORIGINAL_IMAGES_DIR = os.path.join(BASE_DIR, "images")  # Contains full-res orthomosaics

# Full-sized data directories
FULL_ORTHOMOSAICS_DIR = ORIGINAL_IMAGES_DIR  # Same as images
FULL_MASKS_DIR = os.path.join(BASE_DIR, "masks")
FULL_HEIGHTMAPS_DIR = os.path.join(BASE_DIR, "heightmaps")
FULL_NORMALMAPS_DIR = os.path.join(BASE_DIR, "normalmaps")

# Snippet directories
SNIPPET_ORTHOMOSAICS_DIR = os.path.join(BASE_DIR, "snippets_orthomosaics")
SNIPPET_MASKS_DIR = os.path.join(BASE_DIR, "snippets_masks")
SNIPPET_NORMALMAPS_DIR = os.path.join(BASE_DIR, "snippets_normalmaps")

# Parameters
CROP_SIZE = 1280  # Size of the snippets
DESIRED_COVERAGE = 1.6  # Coverage factor for Sobol splitting (1.6 = 160% coverage)
BLACK_THRESHOLD = 1.0  # Threshold for excluding black images (1.0 = 100% black pixels)
EPS = 0.001  # Scaling factor for normal map computation

# Class colors for COCO mask generation
CLASS_COLORS = {
    1: (0, 0, 255),      # Class 1 - Blue 
    2: (255, 255, 0),    # Class 2 - Yellow
    3: (255, 0, 0),      # Class 3 - Red
    # 4: (0, 255, 0),    # Class 4 - Green
}

# GLOBAL INDEX FOR SOBOL SPLITTING (DO NOT MODIFY)
GLOBAL_CROP_INDEX = 1

# Create all directories if they don't exist
directories = [
    ORIGINAL_IMAGES_DIR, FULL_MASKS_DIR, FULL_HEIGHTMAPS_DIR, FULL_NORMALMAPS_DIR,
    SNIPPET_ORTHOMOSAICS_DIR, SNIPPET_MASKS_DIR, SNIPPET_NORMALMAPS_DIR
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✓ Directory ready: {directory}")

print("\n✅ All directories created/verified!")

✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\images
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\masks
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\heightmaps
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\normalmaps
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\snippets_orthomosaics
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\snippets_masks
✓ Directory ready: C:/Users/admin/Desktop/AWS_TRAINING/2025-08-10_4classEX\snippets_normalmaps

✅ All directories created/verified!


# 2. Import All Required Libraries

In [9]:
import json
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask
import cv2
import math
import sobol_seq
import rasterio
import glob

# Disable PIL image size limit
Image.MAX_IMAGE_PIXELS = None

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


# 3. Export COCO JSON to Mask Images

In [ ]:
def export_coco_json_to_masks():
    """Export COCO JSON annotations to colored mask images."""
    print("\n=== Step 3: Exporting COCO JSON to Masks ===")
    
    # Load COCO JSON
    with open(COCO_JSON_PATH) as f:
        coco_data = json.load(f)
    
    # COCO API object
    coco = COCO(COCO_JSON_PATH)
    
    # Get all image IDs
    image_ids = coco.getImgIds()
    
    # Process each image - SORTED for consistency
    for i, image_id in enumerate(sorted(image_ids)):
        # Load image data
        img_data = coco.loadImgs(image_id)[0]
        img_filename = img_data['file_name']
        
        # Load annotations for this image
        ann_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(ann_ids)
        
        # Create empty mask
        img_width = img_data['width']
        img_height = img_data['height']
        mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)
        
        # Process each annotation
        for ann in annotations:
            # Get mask for this annotation
            if isinstance(ann['segmentation'], list):  # Polygon
                seg_mask = coco.annToMask(ann)
            else:  # RLE
                rle = ann['segmentation']
                seg_mask = coco_mask.decode(rle)
            
            # Get class color
            class_id = ann['category_id']
            color = CLASS_COLORS.get(class_id, (0, 0, 0))
            
            # Apply color to mask
            mask[seg_mask == 1] = color
        
        # Save mask
        mask_img = Image.fromarray(mask)
        output_path = os.path.join(FULL_MASKS_DIR, img_filename.replace('.jpg', '_mask.png'))
        mask_img.save(output_path)
        
        if i % 10 == 0:
            print(f"  Processed {i+1}/{len(image_ids)} images...")
    
    print(f"✅ Complete: {len(image_ids)} masks saved to {FULL_MASKS_DIR}")

# Run Step 3
export_coco_json_to_masks()

# 4(a) Data Augmentation - Flip PNG Images (Orthomosaics & Masks)

In [10]:
def flip_png_images(input_folder, image_type="images"):
    """Flip PNG images horizontally for data augmentation."""
    print(f"\n=== Step 4a: Flipping {image_type} ===")
    
    processed = 0
    # SORTED for consistency
    for filename in sorted(os.listdir(input_folder)):
        if filename.lower().endswith(".png"):
            file_path = os.path.join(input_folder, filename)
            
            # Skip if already flipped
            if filename.startswith("flipped_"):
                continue
            
            # Open and flip image
            with Image.open(file_path) as img:
                flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
            
            # Save flipped image
            new_filename = "flipped_" + filename
            output_path = os.path.join(input_folder, new_filename)
            flipped_img.save(output_path)
            processed += 1
    
    print(f"✅ Flipped {processed} {image_type}")

# Run flipping for orthomosaics and masks
if os.path.exists(FULL_ORTHOMOSAICS_DIR):
    flip_png_images(FULL_ORTHOMOSAICS_DIR, "orthomosaics")
flip_png_images(FULL_MASKS_DIR, "masks")


=== Step 4a: Flipping orthomosaics ===
✅ Flipped 12 orthomosaics

=== Step 4a: Flipping masks ===
✅ Flipped 12 masks


# 4(b) Data Augmentation - Flip TIFF Images (Heightmaps)

In [11]:
def flip_geotiffs(input_folder):
    """Flip GeoTIFF images horizontally for data augmentation."""
    print(f"\n=== Step 4b: Flipping Heightmaps ===")
    
    processed = 0
    # SORTED for consistency
    for filename in sorted(os.listdir(input_folder)):
        if filename.lower().endswith((".tif", ".tiff")):
            # Skip if already flipped
            if filename.startswith("flipped_"):
                continue
                
            in_fp = os.path.join(input_folder, filename)
            out_fp = os.path.join(input_folder, "flipped_" + filename)
            
            # Read with rasterio
            with rasterio.open(in_fp) as src:
                profile = src.profile.copy()
                data = src.read()  # shape: (bands, height, width)
            
            # Flip horizontally
            flipped_data = np.flip(data, axis=2)
            
            # Write flipped data
            with rasterio.open(out_fp, 'w', **profile) as dst:
                dst.write(flipped_data)
            
            processed += 1
    
    print(f"✅ Flipped {processed} heightmaps")

# Run flipping for heightmaps if directory exists
if os.path.exists(FULL_HEIGHTMAPS_DIR):
    flip_geotiffs(FULL_HEIGHTMAPS_DIR)


=== Step 4b: Flipping Heightmaps ===
✅ Flipped 12 heightmaps


# 5. Convert Heightmaps to Normal Maps

In [12]:
def load_hdr_image(input_path):
    """Load heightmap from TIFF file."""
    with rasterio.open(input_path) as src:
        dem = src.read(1)
    return dem

def compute_normals(dem, eps):
    """Compute normal map from heightmap."""
    dzdx = np.gradient(dem, axis=1)  # Derivative in x-direction
    dzdy = np.gradient(dem, axis=0)  # Derivative in y-direction
    normals = np.dstack((dzdx, dzdy, eps * np.ones_like(dem)))
    norm = np.linalg.norm(normals, axis=2, keepdims=True)
    return normals / norm

def save_normal_map_png(normals, output_path):
    """Save normal map as PNG."""
    normals_scaled = (normals + 1) / 2
    normals_scaled = np.clip(normals_scaled, 0, 1)
    normals_8bit = (normals_scaled * 255).astype(np.uint8)
    image = Image.fromarray(normals_8bit)
    image.save(output_path, format="PNG")

def process_heightmaps_to_normals():
    """Convert all heightmaps to normal maps."""
    print(f"\n=== Step 5: Converting Heightmaps to Normal Maps ===")
    
    if not os.path.exists(FULL_HEIGHTMAPS_DIR):
        print("  No heightmaps directory found, skipping normal map generation")
        return
    
    pattern = os.path.join(FULL_HEIGHTMAPS_DIR, '*.tif')
    files = sorted(glob.glob(pattern))  # SORTED for consistency
    
    if not files:
        print("  No .tif files found in heightmaps directory")
        return
    
    for i, inp in enumerate(files):
        base = os.path.splitext(os.path.basename(inp))[0]
        outp = os.path.join(FULL_NORMALMAPS_DIR, f"{base}_normalmap.png")
        
        # Process heightmap to normal map
        dem = load_hdr_image(inp)
        normals = compute_normals(dem, EPS)
        save_normal_map_png(normals, outp)
        
        if i % 5 == 0:
            print(f"  Processed {i+1}/{len(files)} heightmaps...")
    
    print(f"✅ Converted {len(files)} heightmaps to normal maps")

# Run heightmap to normal conversion
process_heightmaps_to_normals()


=== Step 5: Converting Heightmaps to Normal Maps ===
  Processed 1/24 heightmaps...
  Processed 6/24 heightmaps...
  Processed 11/24 heightmaps...
  Processed 16/24 heightmaps...
  Processed 21/24 heightmaps...
✅ Converted 24 heightmaps to normal maps


# 6. Sobol Splitting - WITH CONTINUOUS GLOBAL NUMBERING

**The numbering continues across ALL images**:
- Image 1: crops 00000001-00000345
- Image 2: crops 00000346-00000690
- Image 3: crops 00000691-00001234
- etc.

In [13]:
def reset_global_index():
    """Reset the global index counter before starting the splitting process."""
    global GLOBAL_CROP_INDEX
    GLOBAL_CROP_INDEX = 1
    print(f"  Global index counter reset to 1")

def sobol_split_images_continuous(input_folder, output_folder, data_type="images"):
    """
    Split images using Sobol sequence sampling with CONTINUOUS global numbering.
    The index continues across ALL images, not restarting for each image.
    """
    global GLOBAL_CROP_INDEX  # Use the global counter
    
    print(f"\n=== Step 6: Sobol Splitting {data_type} ===")
    print(f"  Starting from index: {GLOBAL_CROP_INDEX:08d}")
    
    total_processed = 0
    total_saved = 0
    
    # CRITICAL: Sort files to ensure consistent processing order
    files_to_process = sorted([f for f in os.listdir(input_folder) 
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'))])
    
    for file_idx, file in enumerate(files_to_process):
        image_path = os.path.join(input_folder, file)
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:
            print(f"  Error loading {file}, skipping...")
            continue
        
        image_height, image_width = image.shape[:2]
        if image_width < CROP_SIZE or image_height < CROP_SIZE:
            print(f"  {file} is smaller than crop size {CROP_SIZE}, skipping...")
            continue
        
        # Calculate number of Sobol points
        image_area = image_width * image_height
        crop_area = CROP_SIZE * CROP_SIZE
        num_points = math.ceil((DESIRED_COVERAGE * image_area) / crop_area)
        
        # Generate Sobol points
        sobol_points = sobol_seq.i4_sobol_generate(2, num_points)
        sobol_points[:, 0] = (sobol_points[:, 0] * (image_width - CROP_SIZE)).astype(int)
        sobol_points[:, 1] = (sobol_points[:, 1] * (image_height - CROP_SIZE)).astype(int)
        
        base_name, _ = os.path.splitext(file)
        crops_from_this_image = 0
        starting_index = GLOBAL_CROP_INDEX  # Remember where we started for this image
        
        # Save EVERY crop with CONTINUOUS global numbering
        for (x, y) in sobol_points:
            x, y = int(x), int(y)
            cropped_image = image[y:y + CROP_SIZE, x:x + CROP_SIZE]
            
            # Save with CONTINUOUS global index
            output_filename = f"{GLOBAL_CROP_INDEX:08d}_{base_name}.png"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, cropped_image)
            
            total_saved += 1
            crops_from_this_image += 1
            GLOBAL_CROP_INDEX += 1  # INCREMENT THE GLOBAL COUNTER
        
        total_processed += 1
        ending_index = GLOBAL_CROP_INDEX - 1  # Last index used for this image
        print(f"  Processed: {file} → {crops_from_this_image} crops (indices {starting_index:08d}-{ending_index:08d})")
    
    print(f"✅ {data_type}: Processed {total_processed} images, created {total_saved} snippets")
    print(f"  Final index reached: {GLOBAL_CROP_INDEX - 1:08d}")
    return total_saved

# Run Sobol splitting for all three types
print("\n" + "="*80)
print("STEP 6: SOBOL SPLITTING WITH CONTINUOUS GLOBAL NUMBERING")
print("="*80)

counts = {}

# Process orthomosaics
reset_global_index()  # Start from 1
if os.path.exists(FULL_ORTHOMOSAICS_DIR):
    counts['ortho'] = sobol_split_images_continuous(
        FULL_ORTHOMOSAICS_DIR, 
        SNIPPET_ORTHOMOSAICS_DIR, 
        "orthomosaics"
    )

# Process masks - RESET INDEX TO START FROM 1 AGAIN
reset_global_index()  # Start from 1
counts['mask'] = sobol_split_images_continuous(
    FULL_MASKS_DIR, 
    SNIPPET_MASKS_DIR, 
    "masks"
)

# Process normalmaps - RESET INDEX TO START FROM 1 AGAIN
reset_global_index()  # Start from 1
if os.path.exists(FULL_NORMALMAPS_DIR):
    counts['normal'] = sobol_split_images_continuous(
        FULL_NORMALMAPS_DIR, 
        SNIPPET_NORMALMAPS_DIR, 
        "normalmaps"
    )

# Verification
print("\n" + "="*80)
print("VERIFICATION: Snippet counts BEFORE cleaning")
print("="*80)
for key, count in counts.items():
    print(f"  {key}: {count} snippets")
print("\n⚠️  These counts should be identical at this stage.")
print("  Next step will remove black masks and matching orthos/normals.")


STEP 6: SOBOL SPLITTING WITH CONTINUOUS GLOBAL NUMBERING
  Global index counter reset to 1

=== Step 6: Sobol Splitting orthomosaics ===
  Starting from index: 00000001
  Processed: H_Bf_1-4_png-ortho.png → 734 crops (indices 00000001-00000734)
  Processed: H_Bf_15a_png-ortho.png → 8 crops (indices 00000735-00000742)
  Processed: H_Bf_15b_png-ortho.png → 7 crops (indices 00000743-00000749)
  Processed: H_Bf_15c_png-ortho.png → 8 crops (indices 00000750-00000757)
  Processed: H_Bf_16-19_png-ortho.png → 543 crops (indices 00000758-00001300)
  Processed: H_Bf_22_png-ortho.png → 158 crops (indices 00001301-00001458)
  Processed: H_Bf_28_png-ortho.png → 228 crops (indices 00001459-00001686)
  Processed: H_Bf_38-42_png-ortho.png → 91 crops (indices 00001687-00001777)
  Processed: H_Bf_38-42d_png-ortho.png → 91 crops (indices 00001778-00001868)
  Processed: H_Bf_9-14_png-ortho.png → 345 crops (indices 00001869-00002213)
  Processed: K_Bf_13c_png-ortho.png → 250 crops (indices 00002214-000024

# 7. Clean Snippets - Remove Black Masks and Non-Matching Files

This step:
1. First removes any old-format files from previous runs
2. Detects and removes completely black mask snippets
3. Removes corresponding snippets from orthomosaics and normalmaps directories

In [14]:
def clean_old_format_files(directory):
    """Remove any files that don't follow the 8-digit prefix format."""
    import re
    pattern = re.compile(r'^\d{8}_.*\.png$')
    removed = 0
    
    for filename in os.listdir(directory):
        if not pattern.match(filename):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                removed += 1
    
    if removed > 0:
        print(f"  Removed {removed} old-format files from {directory}")
    return removed

def is_completely_black(image_path):
    """Check if an image is completely black (all pixels are [0,0,0])."""
    image = cv2.imread(image_path)
    if image is None:
        return False
    
    # Check if all pixels are exactly [0,0,0] in BGR
    return np.all(image == 0)

def extract_index_from_filename(filename):
    """Extract the 8-digit index from filename like '00000123_image.png'."""
    if len(filename) >= 8 and filename[:8].isdigit():
        return filename[:8]
    return None

def remove_black_masks_and_sync():
    """Remove black masks and synchronize all snippet directories."""
    print("\n=== Step 7: Cleaning Snippets ===")
    
    # Phase 1: Clean old format files
    print("\n📁 Phase 1: Cleaning old-format files...")
    directories = [SNIPPET_MASKS_DIR, SNIPPET_ORTHOMOSAICS_DIR]
    if os.path.exists(SNIPPET_NORMALMAPS_DIR):
        directories.append(SNIPPET_NORMALMAPS_DIR)
    
    for directory in directories:
        clean_old_format_files(directory)
    
    # Phase 2: Find and remove black masks
    print("\n🔍 Phase 2: Finding black masks...")
    black_mask_indices = set()
    total_masks = 0
    
    for filename in sorted(os.listdir(SNIPPET_MASKS_DIR)):
        if filename.endswith('.png'):
            total_masks += 1
            file_path = os.path.join(SNIPPET_MASKS_DIR, filename)
            
            if is_completely_black(file_path):
                index = extract_index_from_filename(filename)
                if index:
                    black_mask_indices.add(index)
    
    print(f"  Found {len(black_mask_indices)} completely black masks out of {total_masks} total masks")
    
    # Phase 3: Remove black masks and corresponding files
    print("\n🗑️  Phase 3: Removing black masks and corresponding files...")
    
    removed_counts = {'masks': 0, 'orthos': 0, 'normals': 0}
    
    # Remove from masks directory
    for filename in os.listdir(SNIPPET_MASKS_DIR):
        index = extract_index_from_filename(filename)
        if index in black_mask_indices:
            file_path = os.path.join(SNIPPET_MASKS_DIR, filename)
            os.remove(file_path)
            removed_counts['masks'] += 1
    
    # Remove from orthomosaics directory
    if os.path.exists(SNIPPET_ORTHOMOSAICS_DIR):
        for filename in os.listdir(SNIPPET_ORTHOMOSAICS_DIR):
            index = extract_index_from_filename(filename)
            if index in black_mask_indices:
                file_path = os.path.join(SNIPPET_ORTHOMOSAICS_DIR, filename)
                os.remove(file_path)
                removed_counts['orthos'] += 1
    
    # Remove from normalmaps directory
    if os.path.exists(SNIPPET_NORMALMAPS_DIR):
        for filename in os.listdir(SNIPPET_NORMALMAPS_DIR):
            index = extract_index_from_filename(filename)
            if index in black_mask_indices:
                file_path = os.path.join(SNIPPET_NORMALMAPS_DIR, filename)
                os.remove(file_path)
                removed_counts['normals'] += 1
    
    print(f"  Removed {removed_counts['masks']} black masks")
    print(f"  Removed {removed_counts['orthos']} corresponding orthomosaics")
    print(f"  Removed {removed_counts['normals']} corresponding normalmaps")
    
    # Phase 4: Final verification
    print("\n✅ Phase 4: Final verification...")
    final_counts = {}
    final_counts['masks'] = len([f for f in os.listdir(SNIPPET_MASKS_DIR) if f.endswith('.png')])
    if os.path.exists(SNIPPET_ORTHOMOSAICS_DIR):
        final_counts['orthos'] = len([f for f in os.listdir(SNIPPET_ORTHOMOSAICS_DIR) if f.endswith('.png')])
    if os.path.exists(SNIPPET_NORMALMAPS_DIR):
        final_counts['normals'] = len([f for f in os.listdir(SNIPPET_NORMALMAPS_DIR) if f.endswith('.png')])
    
    print("\nFinal snippet counts after cleaning:")
    for key, count in final_counts.items():
        print(f"  {key}: {count} snippets")
    
    # Check if all counts match
    all_counts = list(final_counts.values())
    if len(set(all_counts)) == 1:
        print("\n✅ SUCCESS: All directories have the same number of snippets!")
    else:
        print("\n⚠️  WARNING: Snippet counts don't match! Manual verification needed.")
    
    return final_counts

# Run the cleaning process
final_counts = remove_black_masks_and_sync()


=== Step 7: Cleaning Snippets ===

📁 Phase 1: Cleaning old-format files...

🔍 Phase 2: Finding black masks...
  Found 2839 completely black masks out of 4988 total masks

🗑️  Phase 3: Removing black masks and corresponding files...
  Removed 2839 black masks
  Removed 2839 corresponding orthomosaics
  Removed 2839 corresponding normalmaps

✅ Phase 4: Final verification...

Final snippet counts after cleaning:
  masks: 2149 snippets
  orthos: 2149 snippets
  normals: 2149 snippets

✅ SUCCESS: All directories have the same number of snippets!


# 8. Final Summary

In [ ]:
print("\n" + "="*80)
print("PIPELINE COMPLETE - FINAL SUMMARY")
print("="*80)

print("\n📊 Directory Structure:")
print(f"  Base Directory: {BASE_DIR}")
print("\n  Full-sized data:")
print(f"    • Orthomosaics: {FULL_ORTHOMOSAICS_DIR}")
print(f"    • Masks: {FULL_MASKS_DIR}")
print(f"    • Heightmaps: {FULL_HEIGHTMAPS_DIR}")
print(f"    • Normal maps: {FULL_NORMALMAPS_DIR}")
print("\n  Snippets (1280x1280):")
print(f"    • Orthomosaics: {SNIPPET_ORTHOMOSAICS_DIR}")
print(f"    • Masks: {SNIPPET_MASKS_DIR}")
print(f"    • Normal maps: {SNIPPET_NORMALMAPS_DIR}")

print("\n📈 Processing Parameters:")
print(f"  • Crop size: {CROP_SIZE}x{CROP_SIZE} pixels")
print(f"  • Coverage factor: {DESIRED_COVERAGE}x")
print(f"  • Black threshold: {BLACK_THRESHOLD * 100}%")

print("\n✅ All steps completed successfully!")
print("\nYour data is now ready for training.")